In [39]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [40]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
main_dir = 'drive/MyDrive/23Spring-MLMA/isolated/'
os.listdir(main_dir)

['ReadMe', '001', '002', '003', '004', '005', '006']

In [42]:
# get data
def get_original_feature_label(path):
    # scale both vertical and horizontal between 0 and 1
    scaler = MinMaxScaler()
    feature = []
    label = []
    for i in os.listdir(path):
        a = pd.read_csv(str(path+i),names=["vertical","horizontal"])
        a = np.array(a)
        a = scaler.fit_transform(a)
        a = np.swapaxes(a,0,1)
        feature.append(a)
        if i.endswith("001.csv"):
            label.append(0)
        elif i.endswith("002.csv"):
            label.append(1)
        elif i.endswith("003.csv"):
            label.append(2)
        elif i.endswith("004.csv"):
            label.append(3)
        elif i.endswith("005.csv"):
            label.append(4)
        elif i.endswith("006.csv"):
            label.append(5)
        elif i.endswith("007.csv"):
            label.append(6)
        elif i.endswith("008.csv"):
            label.append(7)
        elif i.endswith("009.csv"):
            label.append(8)
        elif i.endswith("010.csv"):
            label.append(9)
        elif i.endswith("011.csv"):
            label.append(10)
        elif i.endswith("012.csv"):
            label.append(11)
    
    return feature,label

# Split on patient level - User-independent


> 6-fold cross-validation





In [24]:
patient = ["001",'002','003','004','005']
train_feature = []
train_label = []
for i in patient:
    f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
    for x in f:
        train_feature.append(x)         
    for x in l:
        train_label.append(x)
print(len(train_feature))
train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))

patient = ["006"]
test_feature = []
test_label = []
for i in patient:
    f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
    for x in f:
        test_feature.append(x)         
    for x in l:
        test_label.append(x)
print(len(test_feature))
test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))

603
121


In [27]:
import xgboost as xgb
# 定义模型
model = xgb.XGBClassifier(objective='multi:softmax', num_class=12)

# 训练模型
model.fit(train_feature, train_label)

# 预测测试集
y_pred = model.predict(test_feature)

# 计算准确率
accuracy = (y_pred == test_label).sum() / len(test_label)
print('Accuracy:', accuracy)

Accuracy: 0.7355371900826446


In [52]:
import xgboost as xgb
def train(train_feature,train_label,test_feature,test_label,num):
    # 定义模型
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=12, random_state=42)

    # 训练模型
    model.fit(train_feature, train_label)

    # 预测测试集
    y_pred = model.predict(test_feature)

    # 计算准确率
    accuracy = (y_pred == test_label).sum() / len(test_label)
    print('Accuracy:', accuracy)

    # 保存模型
    pickle.dump(model, open(main_dir+"XGboost"+str(num)+".pkl", 'wb'))

    return accuracy

In [53]:
patient_all = ["001",'002','003','004','005','006']
acc = []
for j in range(6):
  patient = [patient_all[j]]
  test_feature = []
  test_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          test_feature.append(x)         
      for x in l:
          test_label.append(x)
  print(patient)
  print(len(test_feature))
  test_feature = np.reshape(test_feature,(np.shape(test_feature)[0],-1))

  patient = [x for x in patient_all if x != patient_all[j]]
  print(patient)
  train_feature = []
  train_label = []
  for i in patient:
      f,l = get_original_feature_label(str(main_dir+i+"/isolated_strokes/"))
      for x in f:
          train_feature.append(x)         
      for x in l:
          train_label.append(x)
  print(len(train_feature))
  train_feature = np.reshape(train_feature,(np.shape(train_feature)[0],-1))
  acc.append(train(train_feature,train_label,test_feature,test_label,j))
  print(acc)

['001']
120
['002', '003', '004', '005', '006']
604
Accuracy: 0.6
[0.6]
['002']
120
['001', '003', '004', '005', '006']
604
Accuracy: 0.7916666666666666
[0.6, 0.7916666666666666]
['003']
122
['001', '002', '004', '005', '006']
602
Accuracy: 0.5409836065573771
[0.6, 0.7916666666666666, 0.5409836065573771]
['004']
121
['001', '002', '003', '005', '006']
603
Accuracy: 0.5867768595041323
[0.6, 0.7916666666666666, 0.5409836065573771, 0.5867768595041323]
['005']
120
['001', '002', '003', '004', '006']
604
Accuracy: 0.6916666666666667
[0.6, 0.7916666666666666, 0.5409836065573771, 0.5867768595041323, 0.6916666666666667]
['006']
121
['001', '002', '003', '004', '005']
603
Accuracy: 0.7355371900826446
[0.6, 0.7916666666666666, 0.5409836065573771, 0.5867768595041323, 0.6916666666666667, 0.7355371900826446]


In [54]:
print(np.mean(acc))

0.6577718315795812
